In [2]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray  
from shapely.geometry import shape
from datetime import timedelta
import pystac_client
import odc.stac
from osgeo import gdal

In [22]:
max_veloc = 33
delta = 30


(delta*33/365.25)

2.7104722792607805

In [3]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')

In [4]:
# --- USER INPUTS (edit here while testing) ---
CLOUD_COVER = 10               # percent max
START_MONTH = 7                # inclusive
STOP_MONTH  = 10               # inclusive
NPAIRS = 3                     # number of pairs per image

aoi = {
    "type": "Polygon",
    "coordinates": [
        [ [-121.76644001937807,46.83837147698088],
            [-121.6594983841296,46.83837147698088],
            [-121.6594983841296,46.8948204721259],
            [-121.76644001937807,46.8948204721259],
            [-121.76644001937807,46.83837147698088]]
    ]
}

# --- pystac-client 0.8.6: open client (no session arg here)
stac = pystac_client.Client.open("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")

In [6]:
search = stac.search(
    collections=["HLSS30.v2.0", "HLSL30.v2.0"],
    intersects=aoi,
    query={"eo:cloud_cover": {"lt": 10}},
    max_items=2000,
)

items = search.item_collection()
print(f"Found {len(items)} HLS items.")

Found 1535 HLS items.


In [7]:
crs_utm = gpd.GeoDataFrame({'geometry':[shape(aoi)]}, crs="EPSG:4326").estimate_utm_crs()

In [8]:
aoi_gpd = gpd.GeoDataFrame({'geometry':[shape(aoi)]}, crs="EPSG:4326")
bbox = aoi_gpd.total_bounds

ds = odc.stac.load(
    items,
    chunks={"x": 2048, "y": 2048},
    bbox=bbox,
    resolution=30,
    crs=crs_utm.to_string(),
    groupby="solar_day"
).where(lambda x: x > 0, other=np.nan)

print(f"Loaded {len(ds.time)} acquisitions.")

Loaded 530 acquisitions.


In [9]:
total_pixels = len(ds.y)*len(ds.x)
nan_count = (~np.isnan(ds.B04)).sum(dim=['x', 'y']).compute()

/mnt/working/brencher/sw/miniconda3/envs/image-correlation/lib/python3.13/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/mnt/working/brencher/sw/miniconda3/envs/image-correlation/lib/python3.13/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


In [10]:
ds = ds.where(nan_count >= total_pixels*0.9, drop=True)

In [12]:
ds_snowoff = ds.where((ds.time.dt.month >= 7) & (ds.time.dt.month <= 10), drop=True)

In [18]:
ds_snowoff

<xarray.Dataset> Size: 1GB
Dimensions:      (time: 136, y: 214, x: 276)
Coordinates:
  * y            (y) float64 2kB 5.194e+06 5.194e+06 ... 5.188e+06 5.188e+06
  * x            (x) float64 2kB 5.94e+05 5.94e+05 ... 6.022e+05 6.022e+05
    spatial_ref  int32 4B 32610
  * time         (time) datetime64[ns] 1kB 2013-07-03T18:57:38.020000 ... 202...
Data variables: (12/33)
    browse       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    thumbnail_0  (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    thumbnail_1  (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    VZA          (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    SZA          (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    B03          (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    ...           ...
    s3_VAA       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    s3_B06       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    s3_Fmask     (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    s3_B04       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    s3_B01       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>
    s3_B10       (time, y, x) float32 32MB dask.array<chunksize=(1, 214, 276), meta=np.ndarray>

In [17]:
# --- Build rolling pairs with <= 100 days baseline ---
pairs = []
for r in range(len(ds_snowoff.time) - int(NPAIRS)):
    for s in range(1, int(NPAIRS) + 1):
        t0 = pd.to_datetime(ds_snowoff.isel(time=r).time.values)
        t1 = pd.to_datetime(ds_snowoff.isel(time=r+s).time.values)
        if (t1 - t0).days <= 100:
            img1_date = image_dates[r]
            img2_date = image_dates[r+s]
            shortname = f"{img1_date}_{img2_date}"
            pairs.append({"img1_date": img1_date, "img2_date": img2_date, "name": shortname})

matrixJSON = json.dumps({"include": pairs})
print(f"number of image pairs: {len(pairs)}")

# If you want to emulate writing to GitHub Actions env:
# os.environ['GITHUB_OUTPUT'] must exist in Actions. In a notebook, just show the values:
print("IMAGE_DATES =", image_dates[:10], "…" if len(image_dates) > 10 else "")
print("MATRIX_PARAMS_COMBINATIONS = (json shown below)")
print(matrixJSON[:500] + ("…" if len(matrixJSON) > 500 else ""))

number of image pairs: 327
IMAGE_DATES = ['2013-07-03', '2013-08-13', '2013-08-20', '2013-09-14', '2014-07-06', '2014-07-15', '2014-08-23', '2014-10-03', '2015-07-02', '2015-07-18'] …
MATRIX_PARAMS_COMBINATIONS = (json shown below)
{"include": [{"img1_date": "2013-07-03", "img2_date": "2013-08-13", "name": "2013-07-03_2013-08-13"}, {"img1_date": "2013-07-03", "img2_date": "2013-08-20", "name": "2013-07-03_2013-08-20"}, {"img1_date": "2013-07-03", "img2_date": "2013-09-14", "name": "2013-07-03_2013-09-14"}, {"img1_date": "2013-08-13", "img2_date": "2013-08-20", "name": "2013-08-13_2013-08-20"}, {"img1_date": "2013-08-13", "img2_date": "2013-09-14", "name": "2013-08-13_2013-09-14"}, {"img1_date": "2013-08-20", "img2_date": "…
